In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow_datasets as tfds
from tensorflow.keras.applications import EfficientNetB4

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
efficienNet = EfficientNetB4(weights="imagenet", include_top=False, input_shape=(256,256,3))

In [36]:
print(efficienNet.layers[326].output.shape)
print(efficienNet.layers[149].output.shape)
print(efficienNet.layers[90].output.shape)
print(efficienNet.layers[31].output.shape)

(None, 16, 16, 960)
(None, 32, 32, 336)
(None, 64, 64, 192)
(None, 128, 128, 144)


In [21]:
def encoder_subblock(efNet_conection_output):
    x = layers.Conv2D(3,3)(efNet_conection_output)
    x = residual_block(x)
    x = residual_block(x)
    x = layers.LeakyReLU()(x)
    return x


def efNet_conection(efNet_output):
    x1 = layers.LeakyReLU()(efNet_output)
    x2 = layers.MaxPool2D((2,2))(x1)
    x3 = layers.Dropout(0.3)(x2)
    return x3


def decoder_subblock(encoder_output, prev_layer):
    unification = layers.concatenate([encoder_output, prev_layer])
    x1 = layers.Dropout(0.3)(unification)
    x2 = layers.Conv2D(3,3)(x)
    x3 = residual_block(x2)
    x3 = residual_block(x3)
    x4 = layers.LeakyReLU()(x3)
    return x4


def residual_block(prev_layer):
    path_1 = layers.LeakyReLU()(prev_layer)
    path_1 = layers.BatchNormalization()(path_1)
    path_1 = layers.Conv2D(3,3)(path_1)
    path_1 = layers.BatchNormalization()(path_1)
    path_1 = layers.LeakyReLU()(path_1)
    path_1 = layers.Conv2D(3,3)(path_1)
    path_1 = layers.BatchNormalization()(path_1)
    path_2 = layers.BatchNormalization()(prev_layer)
    return layers.concatenate([path_1, path_2])

    
def up_sampling(prev_layer, deep):
    x = layers.Conv2DTranspose(deep, 2)(prev_layer)
    return x


def last_up_sampling(prev_layer):
    x = layers.Conv2D(1,1, padding='same', activation='Sigmoid')(prev_layer)
    return x


In [37]:
deep1 = efficienNet.layers[31].output.shape[3]
deep2 = efficienNet.layers[90].output.shape[3]
deep3 = efficienNet.layers[149].output.shape[3]
deep4 = efficienNet.layers[326].output.shape[3]

In [40]:
def build_unet_model():
     # inputs
    inputs = layers.Input(shape=(256,256,1))
    
    adaptation = layers.Conv2D(3, 3, padding="same", activation = "elu")(inputs)
    
    # ENCONDING
    # (128,128,144)
    dw1 = efficienNet.layers[31].output
    # (64,64,192)
    dw2 = efficienNet.layers[90].output
    # (32,32,336)
    dw3 = efficienNet.layers[149].output
    # (16,16,960)
    dw4 = efficienNet.layers[326].output

    # Encoder sub block (8,8,960)
    middle = efNet_conection(dw4)
    
    # DECODING
    uc1 = up_sampling(middle, deep4)
    up1 = decoder_subblock(dw4, uc1)
    
    uc2 = up_sampling(up1)
    up1 = decoder_subblock(dw3, uc2)
    
    uc3 = up_sampling(up2)
    up3 = decoder_subblock(dw2, uc3)
    
    uc4 = up_sampling(up3)
    up4 = decoder_subblock(dw1, uc4)

    uc5 = up_sampling(up4)

    # outputs
    outputs = last_up_sampling(uc5)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model

In [41]:
unet_model = build_unet_model()

ResourceExhaustedError: OOM when allocating tensor with shape[4,4,960,960] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]